In [1]:
"""
This cell implements the full MIDI-to-53TET conversion logic, including a comprehensive
Chord Mapping engine that translates 12-TET chord qualities into their 53-TET equivalents.
"""
import numpy as np
from pathlib import Path
import mido
import sys
import ast
import traceback
import os

# --- CONVENTION MODULE IMPORT ---
try:
    from src import convention
    print("✅ Successfully imported src.convention")
except ImportError:
    try:
        import convention
        print("✅ Successfully imported convention")
    except ImportError:
        # Fallback for when running from root without proper package context
        sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))
        try:
            import convention
            print("✅ Imported convention after path adjust")
        except ImportError:
            print("❌ CRITICAL: convention module not found. Please ensure src/convention.py exists.")

# 53-TET Note Definitions (0-52)
NOTE_NAMES_53TET = [
    "C", "^C", "^^C", "vvC#", "vC#", "C#", "^C#", "^^C#", "vD", "D", 
    "^D", "^^D", "vvD#", "vD#", "D#", "^^Eb", "vvE", "vE", "E", "^E", 
    "^^E", "vF", "F", "^F", "^^F", "vvF#", "vF#", "F#", "^F#", "^^F#", 
    "vG", "G", "^G", "^^G", "vvG#", "vG#", "G#", "^G#", "vvA", "vA", 
    "A", "^A", "^^A", "vBb", "Bb", "^Bb", "^^Bb", "vvB", "vB", "B", 
    "^B", "^^B", "vC"
]

def identify_semantic_quality(steps):
    """
    Returns the semantic name (e.g. 'supermajor') for the interval step count.
    Uses the convention module's glossary.
    """
    if steps is None: return None
    steps = steps % 53
    # Delegate to convention module
    return convention.STEP_TO_SEMANTIC.get(steps, f"step{steps}")

def get_new_chord_name(intervals_in_steps):
    """
    Delegates naming to the convention module logic.
    """
    q3 = identify_semantic_quality(intervals_in_steps.get('third'))
    q5 = identify_semantic_quality(intervals_in_steps.get('fifth'))
    q7 = identify_semantic_quality(intervals_in_steps.get('seventh'))
    
    # Use the convention's sophisticated naming logic
    return convention.get_name(q3, q5, q7)


# Text processing dependencies
try:
    from src import chord_mapping
    print("Successfully imported src.chord_mapping")
except ImportError:
    try:
        import chord_mapping
        print("Successfully imported chord_mapping")
    except ImportError:
        # If running from src directly without package context
        current_dir = Path.cwd()
        if (current_dir / "chord_mapping.py").exists():
            sys.path.append(str(current_dir))
            import chord_mapping
            print("Imported chord_mapping from current directory")
        else:
            print("Warning: Could not import chord_mapping. Text conversion will still work using internal definitions.")

# Modal scale type definitions
MODAL_SCALE_TYPES = {
    'type_0': {
        'name': 'Major',
        'hc_distances': [0, 9, 9, 4, 9, 9, 9],  # Standard 12-TET
        'description': 'Standard major scale (baseline 12-TET)'
    },
    'type_1': {
        'name': 'Neutral',
        'hc_distances': [0, 8, 7, 7, 9, 8, 7],
        'description': 'Neutral mode with neutral intervals'
    },
    'type_2': {
        'name': 'SubMinor',
        'hc_distances': [0, 9, 3, 10, 9, 3, 10],
        'description': 'Subminor mode'
    },
    'type_3': {
        'name': 'H_3rd_H_7th',
        'hc_distances': [0, 9, 8, 5, 9, 8, 5],
        'description': 'Harmonic 3rd and 7th mode'
    },
    'type_4': {
        'name': 'UpMajor',
        'hc_distances': [0, 10, 9, 3, 9, 10, 9],
        'description': 'Up-major mode'
    },
    'type_5': {
        'name': 'Major_v2',
        'hc_distances': [0, 9, 9, 5, 8, 8, 9],
        'description': 'Alternative major mode'
    },
    'type_6': {
        'name': 'Neutral_N',
        'hc_distances': [0, 8, 7, 7, 9, 5, 10],
        'description': 'Neutral N mode'
    }
}

def get_53tet_ratio(steps):
    return 2 ** (steps / 53.0)

def find_closest_53tet_step(ratio):
    steps = round(53 * np.log2(ratio))
    return steps

def build_chromatic_scale_53tet(hc_distances, root_step=0, tonic_position=0):
    scale_7_steps = np.cumsum(hc_distances)
    relative_positions = [0, 2, 4, 5, 7, 9, 11]
    scale_positions = [(pos + tonic_position) % 12 for pos in relative_positions]
    chromatic_steps = [None] * 12
    for i, pos in enumerate(scale_positions):
        chromatic_steps[pos] = root_step + scale_7_steps[i]
    for i in range(12):
        if chromatic_steps[i] is None:
            prev_pos = -1
            next_pos = 12
            for j in range(i - 1, -1, -1):
                if chromatic_steps[j] is not None:
                    prev_pos = j
                    break
            for j in range(i + 1, 12):
                if chromatic_steps[j] is not None:
                    next_pos = j
                    break
            if prev_pos >= 0 and next_pos < 12:
                prev_step = chromatic_steps[prev_pos]
                next_step = chromatic_steps[next_pos]
                range_steps = next_step - prev_step
                positions = next_pos - prev_pos
                offset = i - prev_pos
                chromatic_steps[i] = round(prev_step + (range_steps * offset / positions))
            elif prev_pos >= 0:
                prev_step = chromatic_steps[prev_pos]
                range_steps = (root_step + 53) - prev_step
                positions = 12 - prev_pos
                offset = i - prev_pos
                chromatic_steps[i] = round(prev_step + (range_steps * offset / positions))
            else:
                chromatic_steps[i] = root_step + round((i / 12) * 53)
    return chromatic_steps

def calculate_53tet_frequency(midi_note, chromatic_scale_steps):
    tet12_freq = 440.0 * (2 ** ((midi_note - 69) / 12))
    note_in_octave = midi_note % 12
    step_53tet = chromatic_scale_steps[note_in_octave]
    step_12tet = (note_in_octave * 53) / 12
    hc_deviation = step_53tet - step_12tet
    ratio = 2 ** (hc_deviation / 53)
    frequency = tet12_freq * ratio
    return frequency

def calculate_pitch_bend_for_frequency(target_freq, midi_note):
    a4_freq = 440.0
    a4_midi = 69
    tet12_freq = a4_freq * (2 ** ((midi_note - a4_midi) / 12))
    if tet12_freq > 0:
        cents = 1200 * np.log2(target_freq / tet12_freq)
    else:
        cents = 0
    return cents

def process_text_file_conversion(input_path, scale_type, key, chromatic_scale, output_dir=None):
    """
    Finds the corresponding text file for a MIDI file and converts its chords to 53-TET notation.
    """
    print(f"DEBUG: process_text_file_conversion called for {input_path}")
    
    # Locate text directory
    potential_text_dirs = [
        input_path.parent.parent.parent / "text_files", 
        input_path.parent.parent / "text_files",        
        Path("dataset/text_files").resolve(),
        Path("../dataset/text_files").resolve()
    ]
    
    text_dir = None
    for d in potential_text_dirs:
        if d.exists():
            text_dir = d
            break
            
    if text_dir is None:
        print("⚠️ Could not locate text_files directory.")
        return

    text_filename = input_path.stem + ".txt"
    text_path = text_dir / text_filename
    
    if not text_path.exists():
        text_filename_alt = input_path.stem.split("_type")[0] + ".txt" 
        text_path_alt = text_dir / text_filename_alt
        if text_path_alt.exists():
             text_path = text_path_alt
        else:
            print(f"ℹ️ Corresponding text file not found: {text_path}")
            return
        
    print(f"📄 Processing text file: {text_path.name}")
    
    try:
        with open(text_path, 'r') as f:
            content = f.read()
            try:
                chord_data = ast.literal_eval(content)
            except:
                print("Could not parse text file as list structure")
                return
            
        modified_chords = []
        
        chromatic_map = {
            'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3,
            'E': 4, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8,
            'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
        }
        
        i = 0
        while i < len(chord_data):
            token = chord_data[i]
            
            # Handle Slash chords (Bass note) 
            if token == '/':
                modified_chords.append(token)
                i += 1
                if i < len(chord_data):
                    bass_token = chord_data[i]
                    if bass_token in chromatic_map:
                         bass_idx = chromatic_map[bass_token]
                         bass_step = chromatic_scale[bass_idx]
                         new_bass_name = NOTE_NAMES_53TET[bass_step]
                         modified_chords.append(new_bass_name)
                    else:
                         modified_chords.append(bass_token) 
                    i += 1
                continue

            # Check for Root match
            is_root = False
            root_text = ""
            
            if token in chromatic_map:
                is_root = True
                root_text = token
            
            if not is_root:
                modified_chords.append(token)
                i += 1
                continue
            
            # Found Root
            quality_text = ""
            has_quality_token = False
            current_idx = i
            
            if current_idx + 1 < len(chord_data):
                next_tok = chord_data[current_idx + 1]
                if (next_tok not in ['|', '|:', ':|', 'e||', 'b||', '/', '.'] 
                    and not next_tok.startswith('Form_')
                    and not (next_tok.replace('.','',1).isdigit())): 
                    
                    quality_text = next_tok
                    has_quality_token = True
            
            # Parse Intervals
            input_intervals = {
                'third': 4, 
                'fifth': 7, 
                'seventh': None 
            }
            
            q = quality_text
            if 'maj7' in q or 'Maj7' in q:
                input_intervals['seventh'] = 11
            elif 'maj' in q: 
                pass
            elif 'dom7' in q or q == '7':
                input_intervals['seventh'] = 10
            elif 'm7' in q or 'min7' in q: 
                input_intervals['third'] = 3
                input_intervals['seventh'] = 10
            elif 'm' in q or 'min' in q:
                input_intervals['third'] = 3
                if '7' in q: input_intervals['seventh'] = 10 
            elif 'dim' in q or 'ø' in q:
                input_intervals['third'] = 3
                input_intervals['fifth'] = 6
                if '7' in q: input_intervals['seventh'] = 9 
                if 'ø' in q: input_intervals['seventh'] = 10 
            elif 'aug' in q or '+' in q:
                input_intervals['fifth'] = 8
                if '7' in q: input_intervals['seventh'] = 10
                
            # Calculate 53-TET steps
            root_idx_12 = chromatic_map[root_text]
            root_step = chromatic_scale[root_idx_12]
            
            def get_step_from_12tet_interval(root_step, interval_12):
                target_idx_12 = (root_idx_12 + interval_12) % 12
                target_step = chromatic_scale[target_idx_12]
                diff = target_step - root_step
                if diff < 0: diff += 53
                return diff

            steps_map = {}
            steps_map['third'] = get_step_from_12tet_interval(root_step, input_intervals['third'])
            steps_map['fifth'] = get_step_from_12tet_interval(root_step, input_intervals['fifth'])
            
            if input_intervals['seventh'] is not None:
                steps_map['seventh'] = get_step_from_12tet_interval(root_step, input_intervals['seventh'])
            else:
                steps_map['seventh'] = None
                
            # Get New Suffix
            new_suffix = get_new_chord_name(steps_map)
            
            new_root_name = NOTE_NAMES_53TET[root_step]
            modified_chords.append(new_root_name)
            modified_chords.append(new_suffix)
            
            if has_quality_token:
                i += 2 
            else:
                i += 1 

        if output_dir is None:
             output_path_dir = input_path.parent
        else:
             output_path_dir = output_dir

        output_text_filename = f"{input_path.stem}_{scale_type}.txt"
        output_text_path = output_path_dir / output_text_filename
        
        with open(output_text_path, 'w') as f:
            f.write(str(modified_chords))
            
        print(f"✅ Text conversion saved: {output_text_path}")
        
    except Exception as e:
        print(f"❌ Error processing text file: {e}")
        traceback.print_exc()

def convert_midi_to_53tet(input_midi_path, scale_type='type_1', output_dir=None, key=None):
    print(f"\n{'='*70}")
    print(f"53-TET MODAL CONVERSION: {scale_type}")
    print(f"{'='*70}\n")
    
    input_path = Path(input_midi_path)
    if scale_type not in MODAL_SCALE_TYPES:
        raise ValueError(f"Unknown scale type: {scale_type}")
    
    if key is None:
        import re
        match = re.search(r'_([A-G][#b]?)_(?:major|minor)', input_path.stem)
        if match:
            key = match.group(1)
        else:
            key = 'C'
            print(f"⚠️  Could not detect key from filename, defaulting to C")
    
    key_to_position = {
        'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3,
        'E': 4, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8,
        'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
    }
    
    tonic_position = key_to_position.get(key, 0)
    config = MODAL_SCALE_TYPES[scale_type]
    hc_distances = config['hc_distances']
    
    print(f"Key: {key} (tonic at chromatic position {tonic_position})")
    print(f"Scale: {config['name']}")
    
    chromatic_scale = build_chromatic_scale_53tet(hc_distances, root_step=0, tonic_position=tonic_position)
    
    mid = mido.MidiFile(input_path)
    mpe_midi = mido.MidiFile(type=mid.type, ticks_per_beat=mid.ticks_per_beat)
    channel_pool = list(range(1, 16))
    channel_index = 0
    
    print("Processing MIDI tracks...")
    
    for track_idx, track in enumerate(mid.tracks):
        new_track = mido.MidiTrack()
        mpe_midi.tracks.append(new_track)
        
        for msg in track:
            if msg.is_meta:
                new_track.append(msg.copy())
            else:
                break
        
        for ch in range(1, 16):
            if ch == 9: continue
            new_track.append(mido.Message('control_change', control=101, value=0, time=0, channel=ch))
            new_track.append(mido.Message('control_change', control=100, value=0, time=0, channel=ch))
            new_track.append(mido.Message('control_change', control=6, value=2, time=0, channel=ch))
            new_track.append(mido.Message('control_change', control=38, value=0, time=0, channel=ch))
            new_track.append(mido.Message('control_change', control=101, value=127, time=0, channel=ch))
            new_track.append(mido.Message('control_change', control=100, value=127, time=0, channel=ch))
        
        active_notes = {}
        
        for msg in track:
            if msg.is_meta: continue
            
            if msg.type == 'note_on' and msg.velocity > 0:
                target_freq = calculate_53tet_frequency(msg.note, chromatic_scale)
                bend_cents = calculate_pitch_bend_for_frequency(target_freq, msg.note)
                mpe_channel = channel_pool[channel_index % len(channel_pool)]
                channel_index += 1
                active_notes[msg.note] = (mpe_channel, msg.time, bend_cents)
                bend_value = int((bend_cents / 200) * 8192)
                bend_value = max(-8192, min(8191, bend_value))
                new_track.append(mido.Message('pitchwheel', pitch=bend_value, time=msg.time, channel=mpe_channel))
                new_track.append(mido.Message('note_on', note=msg.note, velocity=msg.velocity, time=0, channel=mpe_channel))
            
            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                if msg.note in active_notes:
                    mpe_channel, start_time, bend_cents = active_notes[msg.note]
                    new_track.append(mido.Message('note_off', note=msg.note, velocity=msg.velocity if msg.type == 'note_off' else 0, time=msg.time, channel=mpe_channel))
                    del active_notes[msg.note]
                else:
                    new_track.append(msg.copy())
            else:
                new_track.append(msg.copy())
    
    # Process corresponding Text file
    # Pass output_dir to ensure text file lands next to MIDI
    process_text_file_conversion(input_path, scale_type, key, chromatic_scale, output_dir)

    if output_dir is None:
        output_dir = input_path.parent
    else:
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
    
    output_filename = f"{input_path.stem}_{scale_type}.mid"
    output_path = output_dir / output_filename
    
    mpe_midi.save(output_path)
    print(f"✅ Conversion complete! Saved to {output_path}")
    return output_path

✅ Successfully imported convention
Successfully imported chord_mapping


In [2]:
# ═══════════════════════════════════════════════════════════════════════════════
# TEST: Convert "Something" to any type
# ═══════════════════════════════════════════════════════════════════════════════

# Input: "Something" by The Beatles (test case)
input_midi = Path('/Users/david/ANIMA_Data_Formation/dataset/midi_files/mpe/47832_Something_C_major.mid')

scale_type = "type_3"
# Convert
type_output = convert_midi_to_53tet(
    input_midi_path=input_midi,
    scale_type=scale_type,
    output_dir=Path("/Users/david/ANIMA_Data_Formation/dataset/midi_files"),
)

print(f"\n{'='*70}")
print("✅ CONVERSION COMPLETE")
print(f"{'='*70}")
print(f"\nOriginal (12-TET): {input_midi.name}")
print(f"Converted (53-TET type): {type_output.name}")
print(f"Corresponding Text File: {type_output.stem}.txt (check dataset/text_files)")
print(f"\n📁 Location: {type_output.parent}")



53-TET MODAL CONVERSION: type_3

Key: C (tonic at chromatic position 0)
Scale: H_3rd_H_7th
Processing MIDI tracks...
DEBUG: process_text_file_conversion called for /Users/david/ANIMA_Data_Formation/dataset/midi_files/mpe/47832_Something_C_major.mid
📄 Processing text file: 47832_Something_C_major.txt
✅ Text conversion saved: /Users/david/ANIMA_Data_Formation/dataset/midi_files/47832_Something_C_major_type_3.txt
✅ Conversion complete! Saved to /Users/david/ANIMA_Data_Formation/dataset/midi_files/47832_Something_C_major_type_3.mid

✅ CONVERSION COMPLETE

Original (12-TET): 47832_Something_C_major.mid
Converted (53-TET type): 47832_Something_C_major_type_3.mid
Corresponding Text File: 47832_Something_C_major_type_3.txt (check dataset/text_files)

📁 Location: /Users/david/ANIMA_Data_Formation/dataset/midi_files


In [3]:
# Listen to the audio output to verify the conversion!
# ═══════════════════════════════════════════════════════════════════════════════
# ROBUST MPE RENDERER - Correct Timing & Synthesis
# ═══════════════════════════════════════════════════════════════════════════════

import numpy as np
from scipy.io import wavfile
import mido


def render_mpe_to_wav(midi_path, output_wav=None, sample_rate=44100, speed=1.2):
    """
    Renders MPE MIDI to WAV with correct timing, pitch bends, and ADSR envelope.
    Args:
        speed: Playback speed factor (1.0 = original, 1.5 = 50% faster, etc.)
    """
    print(f"Loading {midi_path.name}...")
    mid = mido.MidiFile(midi_path)

    # Storage for note events: (start_time, duration, frequency, velocity)
    note_events = []

    # State tracking
    channel_bends = {i: 0.0 for i in range(16)}
    active_notes = {}

    current_time = 0.0

    # Parse MIDI messages
    for msg in mid:
        current_time += msg.time / speed

        if msg.type == "pitchwheel":
            # Pitch Bend Range: +/- 2 semitones (+/- 200 cents)
            cents = (msg.pitch / 8192.0) * 200.0
            channel_bends[msg.channel] = cents

        elif msg.type == "note_on" and msg.velocity > 0:
            bend_cents = channel_bends.get(msg.channel, 0.0)
            base_freq = 440.0 * (2 ** ((msg.note - 69) / 12.0))
            freq = base_freq * (2 ** (bend_cents / 1200.0))

            active_notes[(msg.channel, msg.note)] = (
                current_time,
                freq,
                msg.velocity / 127.0,
            )

        elif msg.type == "note_off" or (msg.type == "note_on" and msg.velocity == 0):
            key = (msg.channel, msg.note)
            if key in active_notes:
                start_time, freq, vel = active_notes.pop(key)
                duration = current_time - start_time
                if duration > 0.005:
                    note_events.append((start_time, duration, freq, vel))

    if not note_events:
        print("⚠️ No notes found to render!")
        return None

    # --- ADSR Configuration (Natural Decay) ---
    # To fix "not decaying" / "beep" sound:
    # 1. Zero sustain so notes fade out even if held (Piano-like)
    # 2. Longer decay time for natural fade
    attack_time = 0.02  # Fast but soft attack
    decay_time = 1.2  # Long decay (1s) to silence
    sustain_level = 0.0  # No static sustain (prevents "pure tone" drone)
    release_time = 0.95  # Gentle release on note off

    total_duration = max(t + d for t, d, _, _ in note_events) + release_time + 0.5
    print(
        f"Rendering {len(note_events)} notes. Total duration: {total_duration:.2f}s (Speed: {speed}x)"
    )

    # Synthesis
    num_samples = int(total_duration * sample_rate)
    audio = np.zeros(num_samples, dtype=np.float32)

    # Pre-calculate envelope lengths in samples
    att_len = int(attack_time * sample_rate)
    dec_len = int(decay_time * sample_rate)
    rel_len = int(release_time * sample_rate)

    for start, dur, freq, vel in note_events:
        start_idx = int(start * sample_rate)
        gate_len = int(dur * sample_rate)

        # Buffer for this note (Gate + Release)
        total_note_len = gate_len + rel_len
        env = np.zeros(total_note_len, dtype=np.float32)

        # We use a cursor to fill the buffer sequentially to ensure no overlaps overwrite incorrectly
        cursor = 0

        # 1. Attack Phase
        # We attack for att_len, BUT we must stop if the gate ends before attack finishes
        actual_att = min(att_len, gate_len)
        if actual_att > 0:
            env[0:actual_att] = np.linspace(0.0, 1.0, actual_att, endpoint=False)
            cursor += actual_att

        current_val = 1.0
        # If the gate was shorter than the attack, we didn't reach 1.0.
        if gate_len < att_len:
            current_val = float(actual_att) / att_len

        # 2. Decay Phase
        # Only happens if we are still within the gate
        remaining_gate = gate_len - cursor
        if remaining_gate > 0:
            # We decay from current_val down to sustain_level over dec_len
            # But again, the gate might end during decay
            actual_dec = min(dec_len, remaining_gate)

            # Create decay curve
            # Linear interpolation from current_val to sustain_level
            decay_curve = np.linspace(
                current_val, sustain_level, dec_len, endpoint=False
            )

            # Take what fits
            env[cursor : cursor + actual_dec] = decay_curve[:actual_dec]
            cursor += actual_dec

            # Update current value for next stage
            if actual_dec == dec_len:
                current_val = sustain_level
            else:
                current_val = decay_curve[actual_dec - 1]

        # 3. Sustain Phase
        remaining_gate = gate_len - cursor
        if remaining_gate > 0:
            env[cursor : cursor + remaining_gate] = current_val
            cursor += remaining_gate

        # 4. Release Phase
        # Fade from current_val to 0.0 over release_time
        # We start writing at 'gate_len'
        env[gate_len : gate_len + rel_len] = np.linspace(
            current_val, 0.0, rel_len, endpoint=False
        )

        # Make sure we don't go out of bounds of the main audio buffer
        end_idx = start_idx + len(env)
        if end_idx > num_samples:
            # Trim env if it goes past end of audio buffer
            env = env[: num_samples - start_idx]
            end_idx = num_samples

        # Generate Phase-locked Oscillator (Odd harmonics for "clarinet/triangle" generic sound)
        t = np.arange(len(env)) / sample_rate
        p = 2 * np.pi * freq * t

        # Fundamental + Odd harmonics with alternating signs (approximation of triangle/clarinet)
        # sin(x) - 0.11 sin(3x) + 0.04 sin(5x)
        osc = (1.0 * np.sin(p)) - (0.11 * np.sin(3 * p)) + (0.04 * np.sin(5 * p))

        # Add to main buffer
        # Reduced amplitude slightly to prevent summing clipping
        audio[start_idx:end_idx] += osc * env * vel * 0.15

    # Normalize
    peak = np.max(np.abs(audio))
    if peak > 0:
        audio = audio / peak * 0.95

    audio_int16 = (audio * 32767).astype(np.int16)
    if output_wav is None:
        output_wav = midi_path.with_suffix(".wav")

    wavfile.write(output_wav, sample_rate, audio_int16)
    print(f"✅ Audio saved: {output_wav}")
    return output_wav

In [ ]:
# play the audio file
# Run it
wav_file = render_mpe_to_wav(type_output)
print(f"Run in terminal: open '{wav_file}'")

# Play in Notebook
print("\n🎧 Audio Player:")
from IPython.display import Audio, display

display(Audio(wav_file))

Loading 47832_Something_C_major_type_3.mid...
Rendering 825 notes. Total duration: 311.45s (Speed: 1.2x)
